<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/temp2ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

### Mount data from Google Drive

In [2]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


change dir and check contents

In [3]:
cd '/content/drive/Shared drives/CS_GO_Data'


/content/drive/Shared drives/CS_GO_Data


In [4]:
ls

'Application Alogrithm.gdoc'   de_train.png
 cs_cruise.png                 de_tulip.png
'data preprocessing.txt'       dont_use/
 de_cache.png                  esea_master_dmg_demos.part1.csv
 de_cbble.png                  esea_master_dmg_demos.part2.csv
 de_coast.png                  esea_master_grenades_demos.part1.csv
 de_dust2.png                  esea_master_grenades_demos.part2.csv
 de_empire.png                 esea_master_kills_demos.part1.csv
 de_inferno.png                esea_master_kills_demos.part2.csv
 de_mikla.png                  esea_meta_demos.part1.csv
 de_mirage.png                 esea_meta_demos.part2.csv
 de_new_nuke.png              'final_processed data'/
 de_nuke.png                   map_data.csv
 de_overpass.png               maps/
 de_royal.png                  processed_data/
 de_santorini.png              Updated_CS_GO_Analysis.ipynb
 de_season.png


### loaders

In [5]:
# Function to combine the dataframes and return a single dataframe
def combine_dataframes(df1,df2):
  df_to_return = df1.append(df2)
  df_to_return = df_to_return.reset_index(drop=True)
  return df_to_return

In [6]:
#load raw data
def load_raw_dmg():
  df1 = pd.read_csv('esea_master_dmg_demos.part1.csv')
  df2 = pd.read_csv('esea_master_dmg_demos.part2.csv')
  return combine_dataframes(df1,df2)

def load_raw_grenades():
  df1 = pd.read_csv('esea_master_grenades_demos.part1.csv')
  df2 = pd.read_csv('esea_master_grenades_demos.part2.csv')
  return combine_dataframes(df1,df2) 
  
def load_raw_kills():
  df1 = pd.read_csv('esea_master_kills_demos.part1.csv')
  df2 = pd.read_csv('esea_master_kills_demos.part2.csv')
  return combine_dataframes(df1,df2) 
  
def load_raw_meta():
  df1 = pd.read_csv('esea_meta_demos.part1.csv')
  df2 = pd.read_csv('esea_meta_demos.part2.csv')
  return combine_dataframes(df1,df2)

def load_raw_map_data():
  return pd.read_csv('map_data.csv')

def load_raw_all():
 return load_raw_dmg(), load_raw_grenades(), load_raw_kills(), load_raw_meta(), load_raw_map_data()

### load all data

In [7]:
%%time
raw_damage, raw_grenades, raw_kills, raw_meta, raw_map_data = load_raw_all()

CPU times: user 52.6 s, sys: 6.9 s, total: 59.5 s
Wall time: 1min 16s


In [8]:
#print the shape of the data
print('damage data:',raw_damage.shape)
print('grenades data:',raw_grenades.shape)
print('kills data:',raw_kills.shape)
print('meta data:',raw_meta.shape)
print('map data:',raw_map_data.shape)

damage data: (10538182, 23)
grenades data: (5246458, 23)
kills data: (2742646, 13)
meta data: (377629, 10)
map data: (7, 7)


### 1) Clean the raw data of corrupted data:

We noticed that in some of the lines of the kaggle dataset, the attacking side (att_side) and the victim side (vic_side) don't match the attacking team (att_team) and the victim team(vic_team). This inconsistency poisons not only that line, but also the round, and the entire match as a whole. Therefore, for any matches these corrupt lines should be removed from the dataset.

In [9]:
#identify and output a list of corrupt matches for given the damage dataframe
def find_inconsistent_games(df):
  #removal condition #1: team mismatch when the sides match
  mismatched_teams = raw_damage[(raw_damage.att_side == raw_damage.vic_side ) & (raw_damage.att_team != raw_damage.vic_team )]
  #removal condition #2: side mismatch when the teams match
  mismatched_sides = raw_damage[(raw_damage.att_side != raw_damage.vic_side ) & (raw_damage.att_team == raw_damage.vic_team )]

  df = combine_dataframes(mismatched_teams,mismatched_sides)
  strange_files = df['file'].unique()

  #other erronous matches that we identified 
  strange_files = np.append(strange_files, 'esea_match_13787633.dem') 
  strange_files = np.append(strange_files, 'esea_match_13784811.dem')

  strange_files = np.unique(strange_files)
  return strange_files


In [13]:
#removes the matches specified in match_list from the given list of dataframes
def remove_matches(inp_data, match_list):
  filtered_data = []
  for df in inp_data:
    filtered_data.append(df[~df['file'].isin(strange_files)])

  return filtered_data

In [14]:
data = [raw_damage, raw_grenades, raw_kills, raw_meta]
strange_files = find_inconsistent_games(raw_damage)

print('Percentage of matches removed:',(len(strange_files)/len(raw_meta))*100,'%')

data = remove_matches(data, strange_files)
# for df in data:
#   df = df[~df['file'].isin(strange_files)]
  #filtered.to_csv(f'/content/drive/Shared drives/CS_GO_Data/processed_data/filtered_{target}_demos.csv')

Percentage of matches removed: 0.3868876595812292 %


In [15]:
print('damage data:',data[0].shape)
print('grenades data:',data[1].shape)
print('kills data:',data[2].shape)
print('meta data:',data[3].shape)
print('map data:',raw_map_data.shape)

damage data: (9505465, 23)
grenades data: (4722314, 23)
kills data: (2477355, 13)
meta data: (340333, 10)
map data: (7, 7)


### 2) Anonymize the data: 

A small, but significant number of the matches record the team names of the teams. Replacing the team names with the more generic 'Team 1' and 'Team 2' Tags would maintain the anonymity of the data while removing the possibility of displaying inappropriate and confusing team names.

There are three sources of damage in this dataset:

this is represented by the variable att_side (attacker side). 

1) att_side = <"Enemy team's name"> :
When an individual takes damage from the actions of the enemy team member)

2) att_side = <"Own team's name"> :
In cases of friendly fire or self damage)

3) att_side = "World" :
When an individual takes damage from non-player objects present in the game. Examples include fall damage and collision with moving objects (if any). 

In [17]:
def remove_other_dmg_sources(df):
  #we need to temporarily remove the world damage to create a list of non-standard team namesdef delete_world_damage(df_esea_master_dmg_demos):
  only_enemy_dmg = df[df.att_side != 'None']
  
  #remove friendly/self damage records
  only_enemy_dmg = only_enemy_dmg[only_enemy_dmg.att_side != only_enemy_dmg.vic_side]
  
  #reset index
  only_enemy_dmg = only_enemy_dmg.reset_index(drop=True)
  return only_enemy_dmg

In [20]:
data[0]

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,bomb_site,hitbox,wp,wp_type,att_id,att_rank,vic_id,vic_rank,att_pos_x,att_pos_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,14372,111.8476,World,Animal Style,None,CounterTerrorist,1,0,False,NaN,Generic,Unknown,Unkown,0,0,76561198055054795,0,0.0000,0.00000,0.0000,0.000000
1,esea_match_13770997.dem,1,15972,124.3761,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,18,9,False,NaN,Stomach,USP,Pistol,76561198048742997,0,76561198082200410,0,-1499.6900,63.33829,-669.5558,-79.769570
2,esea_match_13770997.dem,1,16058,125.0495,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,100,0,False,NaN,Head,USP,Pistol,76561198055054795,0,76561197961009213,0,-1066.8740,3.44563,-614.1868,-91.707770
3,esea_match_13770997.dem,1,16066,125.1121,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,12,7,False,NaN,RightArm,Glock,Pistol,76561198082200410,0,76561198055054795,0,-747.3146,-49.32681,-1065.5560,9.381622
4,esea_match_13770997.dem,1,16108,125.4410,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,15,7,False,NaN,Chest,USP,Pistol,76561198048742997,0,76561198082200410,0,-1501.8610,49.19798,-748.4188,-53.469220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10538177,esea_match_13829173.dem,22,243361,1901.4210,Team 1,Team 2,Terrorist,CounterTerrorist,20,2,False,NaN,Stomach,AK47,Rifle,76561197986109746,0,76561198047370175,0,-1132.0250,-191.14730,-1220.2790,-1279.492000
10538178,esea_match_13829173.dem,22,243373,1901.5150,Team 1,Team 2,Terrorist,CounterTerrorist,26,3,False,NaN,Chest,AK47,Rifle,76561197986109746,0,76561198047370175,0,-1132.0250,-191.14730,-1218.2490,-1275.254000
10538179,esea_match_13829173.dem,22,243405,1901.7650,Team 2,Team 1,CounterTerrorist,Terrorist,100,15,False,NaN,Head,AK47,Rifle,76561198047370175,0,76561197986109746,0,-1205.9710,-1271.50400,-1132.0250,-191.147300
10538180,esea_match_13829173.dem,22,243719,1904.2230,Team 2,Team 1,CounterTerrorist,Terrorist,27,4,False,NaN,Chest,AK47,Rifle,76561198047370175,0,76561198103856560,0,-1217.5950,-1091.54300,-1373.8790,-931.702600


In [22]:
def find_rename_mask(df):
  #get the first enemy damage instance of each match. This row has the all the data we are looking for
  to_rename = df.groupby('file').first()

  #only select the relevant columns
  to_rename = to_rename[['att_team', 'vic_team','att_side','vic_side']] 

  # filter out rows(in this case, matches) where team names are the default 'Team 1' and 'Team 2' #only consider games where the team names aren't standard. Check both columns just in case
  to_rename = to_rename[((to_rename.att_team != 'Team 1') & (to_rename.att_team != 'Team 2')) | ((to_rename.vic_team != 'Team 1') & (to_rename.vic_team != 'Team 2'))]

  to_rename.reset_index(inplace=True)

  #create new columns for each default team name
  to_rename['Team 1'] = None
  to_rename['Team 2'] = None

  for index,row in to_rename.iterrows():
    if row['att_side'] == 'CounterTerrorist':
      row['Team 1'] = row['att_team']
      row['Team 2'] = row['vic_team']
    else:
      row['Team 1'] = row['vic_team']
      row['Team 2'] = row['att_team']

  to_rename.drop(columns=['att_team','vic_team','att_side','vic_side'], inplace=True) #drop the other columns #we only need file, Team1, Team2

  return to_rename

In [ ]:
only_enemy_dmg = remove_other_dmg_sources(data[0])
to_rename = find_rename_mask(only_enemy_dmg)
print('number of games that have custom team names: ', len(to_rename))

In [21]:
data[0]

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,bomb_site,hitbox,wp,wp_type,att_id,att_rank,vic_id,vic_rank,att_pos_x,att_pos_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,14372,111.8476,World,Animal Style,None,CounterTerrorist,1,0,False,NaN,Generic,Unknown,Unkown,0,0,76561198055054795,0,0.0000,0.00000,0.0000,0.000000
1,esea_match_13770997.dem,1,15972,124.3761,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,18,9,False,NaN,Stomach,USP,Pistol,76561198048742997,0,76561198082200410,0,-1499.6900,63.33829,-669.5558,-79.769570
2,esea_match_13770997.dem,1,16058,125.0495,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,100,0,False,NaN,Head,USP,Pistol,76561198055054795,0,76561197961009213,0,-1066.8740,3.44563,-614.1868,-91.707770
3,esea_match_13770997.dem,1,16066,125.1121,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,12,7,False,NaN,RightArm,Glock,Pistol,76561198082200410,0,76561198055054795,0,-747.3146,-49.32681,-1065.5560,9.381622
4,esea_match_13770997.dem,1,16108,125.4410,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,15,7,False,NaN,Chest,USP,Pistol,76561198048742997,0,76561198082200410,0,-1501.8610,49.19798,-748.4188,-53.469220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10538177,esea_match_13829173.dem,22,243361,1901.4210,Team 1,Team 2,Terrorist,CounterTerrorist,20,2,False,NaN,Stomach,AK47,Rifle,76561197986109746,0,76561198047370175,0,-1132.0250,-191.14730,-1220.2790,-1279.492000
10538178,esea_match_13829173.dem,22,243373,1901.5150,Team 1,Team 2,Terrorist,CounterTerrorist,26,3,False,NaN,Chest,AK47,Rifle,76561197986109746,0,76561198047370175,0,-1132.0250,-191.14730,-1218.2490,-1275.254000
10538179,esea_match_13829173.dem,22,243405,1901.7650,Team 2,Team 1,CounterTerrorist,Terrorist,100,15,False,NaN,Head,AK47,Rifle,76561198047370175,0,76561197986109746,0,-1205.9710,-1271.50400,-1132.0250,-191.147300
10538180,esea_match_13829173.dem,22,243719,1904.2230,Team 2,Team 1,CounterTerrorist,Terrorist,27,4,False,NaN,Chest,AK47,Rifle,76561198047370175,0,76561198103856560,0,-1217.5950,-1091.54300,-1373.8790,-931.702600
